# Start Up: 
Before Running the cell below, you must ensure that these have been run in Terminal **IN ORDER** : 
- conda update -n base -c defaults conda 

    - cd SageMaker
    
      - cd yelp-dataset-challenge-1-ds
      
         - conda env create -f environment.yml
          
            - source activate ydc1 
                
                - pip install python-decouple
                  
                  - pip install pprintpp
                  
# Spacy Installs: 

   - python -m spacy download en_core_web_lg

        - python -m spacy link en_core_web_lg en

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import s3
from pprintpp import pprint as pp
from sklearn.externals import joblib
nlp = spacy.load('en')

# Load in Bucket
bucket = s3.Bucket('yelpchallenge1')
# Look inside the bucket.
bucket.contents

['API/',
 'API/api.py',
 'API/api_exploration.ipynb',
 'Environments/',
 'Environments/environment.yml',
 'Flask_App/',
 'Flask_App/Pipfile',
 'Flask_App/__init__.py',
 'Flask_App/app.py',
 'Flask_App/models.py',
 'Flask_App/yelp.py',
 'Model/',
 'Model/vect_1.sav',
 'datasets/',
 'datasets/df.csv',
 'datasets/dtm.csv',
 'datasets/dtm_final.csv',
 'datasets/user.json',
 'notebooks/',
 'notebooks/data_cleanup.ipynb',
 'notebooks/official_NB.ipynb',
 'notebooks/vectorization_exploration.ipynb',
 'notebooks/yelp_data_initial_exploration.ipynb']

In [5]:
                    ### ***** DO NOT RUN. ******* #### 
                  ### ***** ALREADY INSTALLED. ****** ###

# Only have to run this once.
# Installs the .csv 'Locally' on SageMaker Instance

#bucket.get('datasets/df.csv', 'df.csv')

# Installing user.json 'Locally'
#bucket.get('datasets/user.json', 'user.json')


    # Load in Bucket
# bucket = s3.Bucket('yelpchallenge1')
    # Look inside the bucket.
# bucket.contents

# Getting Started: Imports

In [6]:
    # Read-in df.csv
df = pd.read_csv('df.csv')

    # Read-in dtm_final.csv (FINAL)
dtm_final = pd.read_csv('dtm_final.csv')

    # import Vectorizer Model
vect2 = joblib.load('vect_2.sav')

In [4]:
# Checking Null Values and Shape
print(df.isna().sum())
print(df.shape)
print(df.dtypes)

stars          0
text           0
date           0
total_votes    0
tokens         0
dtype: int64
(6685874, 5)
stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object


# Model Prep: 

In [42]:
# Imports: 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
# Creating Training Data. 
    # X_train will include All 135,000 Rows, for 773 Vectorized Words(Columns)
    # y_train or the Target Variable  will include all 135,000 Rows, for the stars Column.     
X_train = dtm_final.iloc[:, 0:773]
y_train = dtm_final.iloc[:, 773:774]

In [ ]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfr = RandomForestRegressor()


# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

#Tuning
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,10000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=5, verbose=1)
grid_search.fit(X_train, y_train)